In [1]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
df = pd.read_excel("total_25_final.xlsx")
df.shape

(84102, 15)

In [3]:
class Seperate:
    def __init__(self, df):
        self.df = df
     
    def this_month(self):
        return self.df[self.df.time.str.contains("초|분|시간|일")].reset_index(drop=True)

# BPE 단어학습

In [4]:
# a = Seperate(df)
# this_month = a.this_month()
# this_month.shape

In [5]:
# 일년 내 자료로 title 단어 학습 

with open ('sample_year.txt', 'w', encoding='utf8') as f:
    for row in df.itertuples():
        f.write(row.title)
        f.write('\n')
        
from tokenizers import CharBPETokenizer
bpe = CharBPETokenizer(lowercase=True)
bpe.train(files='sample_year.txt', min_frequency=30, vocab_size=50000) 

## 토큰화 결과 비교

In [6]:
# BPE를 활용한 토크나이저 함수 
# 길이가 1 이상인 것만 추출 
def tokenizer(text):
    enc = bpe.encode(text)
    for token in enc.tokens:
        token = token.replace('</w>', '')
        if len(token) > 1:
            yield token

# 키위 토크나이저 
kiwi = Kiwi()
def extract_noun(text):
    result = kiwi.analyze(text)
    for lemma, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            yield lemma
            
def compare(num):
    text = df.title[num]
    print('article title:' , text)
    print('-'*60)
    print('bpe tokenizer:', list(tokenizer(text)))
    print('kiwi tokenizer:', list(extract_noun(text)))

In [79]:
for i in range(1650,1850):
    compare(i)

# 청정원이 잘 추출됨 

article title: 더바디샵 모링가 세트
------------------------------------------------------------
bpe tokenizer: ['더바디샵', '모링가', '세트']
kiwi tokenizer: ['더바디샵', '모링', '가', '세트']
article title: 마스터 3종 세트
------------------------------------------------------------
bpe tokenizer: ['마스터', '3종', '세트']
kiwi tokenizer: ['마스터', '종', '세트']
article title: 폴란드 볼레스와비예츠 찻잔 2개세트 미사용
------------------------------------------------------------
bpe tokenizer: ['폴란드', '스와', '찻잔', '2개세트', '미사용']
kiwi tokenizer: ['폴란드', '볼레스와비예츠', '찻잔', '개', '세트', '미사']
article title: 록시땅 핸드크림
------------------------------------------------------------
bpe tokenizer: ['록시땅', '핸드크림']
kiwi tokenizer: ['록시', '땅', '핸드', '크림']
article title: 반상기 세트 (미개봉)
------------------------------------------------------------
bpe tokenizer: ['반상기', '세트', '미개봉']
kiwi tokenizer: ['상기', '세트', '개봉']
article title: 현대백화점 한우순우리 국 선물세트
------------------------------------------------------------
bpe tokenizer: ['현대백화점', '한우', '선물세트']
kiwi tokenizer: ['현

kiwi tokenizer: ['포트메리', '식탁', '매트', '개', '판매']
article title: (미개봉)꽁빠니 드 포로방스 핸드워시 ᆞ로션
------------------------------------------------------------
bpe tokenizer: ['미개봉', '핸드워시', '로션']
kiwi tokenizer: ['개봉', '꽁빠', '드', '포로', '방', '핸드워', '시', '로션']
article title: 샤넬 남성용 넥타이
------------------------------------------------------------
bpe tokenizer: ['샤넬', '남성용', '넥타이']
kiwi tokenizer: ['샤넬', '남성', '넥타이']
article title: 일리 커피머신(일리 에스프레소 x7.1)
------------------------------------------------------------
bpe tokenizer: ['커피머신', '에스프레소']
kiwi tokenizer: ['일리', '커피', '머신', '일리', '에스프레소']
article title: 4c 다이아몬드 은목걸이, 귀걸이 셋트 (새상품) 도금처리
------------------------------------------------------------
bpe tokenizer: ['다이아몬드', '목걸이', '귀걸이', '셋트', '새상품']
kiwi tokenizer: ['다이아몬드', '은', '목걸이', '귀걸이', '셋트', '상품', '도금', '처리']
article title: 스틸라 색조화장 세트
------------------------------------------------------------
bpe tokenizer: ['스틸라', '세트']
kiwi tokenizer: ['스틸', '색조', '화장', '세트']
article title: (새상품) 오

In [15]:
compare(14317)

# 핸드케어가 잘 추출됨 

article title: (새상품)데메테르 핸드케어 4종세트
------------------------------------------------------------
bpe tokenizer: ['새상품', '테르', '핸드', '케어', '4종세트']
kiwi tokenizer: ['상품', '데메테르', '핸드케', '종', '세트']


In [17]:
compare(54906)

# 로스팜이 잘 추출됨

article title: [추석선물세트] 롯데의성마늘프리미엄로스팜set 팝니다
------------------------------------------------------------
bpe tokenizer: ['추석선물세트', '롯데', '의성', '마늘', '프리미엄', '로스팜', 'set', '팝니다']
kiwi tokenizer: ['추석', '선물', '세트', '롯데', '성', '마늘', '프리미엄', '로스', '파', 'ᆷ', '파']


In [18]:
compare(4628)

article title: 건면 구포국수 선물세트 (10봉)
------------------------------------------------------------
bpe tokenizer: ['구포', '국수', '선물세트', '10']
kiwi tokenizer: ['건면', '구포', '국수', '선물', '세트', '봉']


# 토큰화 전 키워드 정리

## 딕셔너리 제작을 위한 빈도수 상위 단어 추출

- BPE tokenizer로 결정
- 상품 분류를 위해 선물세트, 새제품과 같은 키워드는 필요하지 않으므로 빈도 상위 단어 중에서 필요한 단어로만 딕셔너리를 제작 

In [22]:
cv = CountVectorizer(max_features=1000, tokenizer=tokenizer)
dtm = cv.fit_transform(df.title)

wc = pd.DataFrame({'단어': cv.get_feature_names(), '빈도':dtm.sum(axis=0).flat})
wc.sort_values('빈도', ascending=True).head(20)

,단어,빈도
467,불가리,69
288,더블,69
382,명품혼합,69
680,올리,69
782,지아,69
643,양수,69
783,진액,69
425,반상기,69
791,참치선물세트,70
456,보리,70


In [21]:
wc.to_excel('wc_1000.xlsx')

## 딕셔너리 제작
- 상품명, 브랜드 명만 추가 
- 상위 1000개 단어 (빈도 70까지) 
- 총 578개의 키워드

In [80]:
# 길이가 1인 것은 따로 추가함 

# 
thanks_dict = {}
thanks_dict["spam"] = ['스팸', '리챔', '스팸선물세트', '로스팜', '스팸세트', '스팸8호', '햄','스팸6호', '스팸3호', '스팸1호', '그릭슈바인','런천미트', '수제햄', '의성마늘', '스팸복합1호', '스팸선물셋트']
thanks_dict["tuna"] = ['동원', '동원참치', '참치', '동원선물세트', '사조', '참치선물세트', '동원참치선물세트', '참치캔', '행호']
thanks_dict["hand"] = ['핸드크림', '핸드', '핸드워시', '손소독제', '더프트앤도프트', '랩신', '레저렉션']
thanks_dict["nuts"] = ['견과류', '견과']
thanks_dict["dried_seafood"] = ['멸치', '광천김', '멸치선물세트']
thanks_dict["oil"] = ['카놀라유', '참기름',  '식용유', '해바라기씨유', '트러플', '들기름', '올리브유', '해바라기유', '고급유', '포도씨유', '올리브오일', '기름', '요리유', '오일세트']
thanks_dict["ginseng"] = ['정관장', '홍삼', '인삼', '홍삼정', '6년근', '다보록', '한삼인', '활기력', '한뿌리', '홍삼스틱', '홍삼원', '홍삼선물세트']
thanks_dict["mix"] = ['청정원', 'cj', '해표', 'cj제일제당', '목우촌', '백설','스팸복합', '현명한선택', 'lg', '튜나리챔', '복합', 'cj선물세트', '안심특선', '특별한선택', '롯데', '특별한', '제일제당', '오뚜기', '농협', '혼합', '현명한', '사조해표', '스팸고급유', '청정원선물세트', '복합1호', '해표선물세트', '씨제이', '풍성한']
thanks_dict["meat"] = ['육포', '한우']
thanks_dict["fruits"] = ['사과', '표고버섯', '과일', '버섯', '곶감', '배']
thanks_dict["snacks"] = ['한과', '초콜릿', '쿠키', '과자']
thanks_dict["spicy"] = ['올리고당', '천일염', '벌꿀', '미엘드', '간장', '식초']
thanks_dict["bath_thanks"] = ['lg생활건강', '아모레퍼시픽', '애경', '생활건강', '엘지생활건강', '아모레', '엘지', '생활의']

#13

gift_dict = {}
gift_dict["hygiene"] = ['마스크', '섬유유연제', '세제']
gift_dict["coffee"] = ['스타벅스', '오설록','차', '커피', '티세트', '커피잔세트', '이디야', '카누', '티백', '드립백', 'tea', '쌍계명차', '에스프레소', '할리스', '맥심', '담터', '투썸', '녹차', '티박스', '네스프레소', '아메리카노', '핸드드립', 'twg', '허브티', '꽃차', '타바론', '홍차', '차세트', '더치커피', '비니스트', '홈카페', '투썸플레이스']
gift_dict["drink"] = ['와인', '신앙촌']
gift_dict["perfume"] = ['양키캔들', '향초', '향수', '캔들', '디퓨저', '조말론', '모링가', '캔들워머', '퍼퓸', '코코도르', '필로우미스트', '방향제', '베르사체', '디퓨져', '딥디크', '아로마', '워머세트', '양키', '바디필로우', '더블유드레스룸', '우드윅', '라벤더', '딥티크', '랑방', '보티브', '자쉐이드', '자캔들', '화이트머스크']
gift_dict["cosmetics"] = ['화장품','자음생', '설화수', '미스트', '크림', '키엘', '튼살크림','프레쉬', '에센스', '헤라', '스킨', '세럼', '토너', '립스틱', '에스티로더', '스킨케어', '비오템', '쿠션', '스틸라', '브러쉬', '탬버린즈', '입생로랑', '달바', '암웨이', '수분크림', '오휘', '디올', '앰플', '꼬달리', 'fresh', '바비브라운', '아이크림', '논픽션', '자음2종', '클렌저', 'ahc', '화장품세트', '미샤', '자음', '마스크팩', '클라란스', '랑콤', '버베나', '코리아나', '틴트', '아이오페', '스크럽', '립밤세트', '베네피트', '기초', '메이크업', '케어', '팔레트', '아이섀도우', '클렌징', '빌리프', '핸드밤', '안나수이', '카밀', '선크림', '갈색병', '이니스프리', '인셀덤', '산타마리아노벨라', '크리니크', '스킨로션', 'lush', '섀도우', '와일드로즈', '프리메라', '기초세트', 'sk2', '립밥', '파운데이션', '핸드크림세트', '한율', '남성화장품', '에멀젼', '공진향', '립세트']
gift_dict["dishes"] = ['커피잔','컵', '한국도자기', '그릇', '찻잔', '머그컵', '접시', '텀블러', '행남자기', '다기세트', '도자기', '포트메리온', '찻잔세트', '그릇세트', '머그', '와인잔', '다기', '식기세트', '티팟', '코렐', '웨지우드', '식기', '광주요', '머그잔', '본차이나', '빌레로이앤보흐', '로얄알버트', '접시세트', '국그릇', '밥그릇', '로얄', '보타닉가든', '유리컵', '로얄포드', '유리잔', '락앤락', '주전자', '맥주잔', '다기셋트', '머그세트', '방짜유기', '밀양도자기', '술잔', '컵세트', '티팟세트', '세라믹', '레녹스', '덴비', '컵받침', '다도세트']
gift_dict["kitchen"] = ['냄비', '수저세트', '수저', '칼세트', '후라이팬', '젓가락', '포크', '냄비세트', '르쿠르제', '밀폐용기', '소주잔', '프라이팬', '티스푼', '커트러리', '테팔', '네오플램', '헹켈', '키친아트', '스텐', '그라인더', '휘슬러', '글라스', '은수저', '르크루제', '스테인레스', '스테인리스', 'wmf', '나이프', '실리콘', '스텐냄비', '주방', '실리트', '반찬통', '스푼', '도마', '조리도구', '글라스락']
gift_dict["man"] = ['남성', '옴므', '남성용', '남자', '포맨', '100사이즈']
gift_dict["jewel"] = ['목걸이', '귀걸이', '스와로브스키', '팔찌', '시계', '반지', '14k', '키링', '진주', '제이에스티나', '악세사리', '24k', '팔찌세트']
gift_dict["cloth"] = ['양말', '원피스', '모자', '파우치', '손수건', '워머', '수건', '가방', '우주복', '지갑', '카드지갑', '속옷', '아디다스', '내의', '잠옷', '나이키', '폴로', '상하세트', '보스턴백', '가디건', '반지갑', '랄프로렌', '상하의', '속옷세트', '팬티', '브라', '벨트', '디스커버리', '장지갑', '양말세트', '내복', '트레이닝복', '넥타이', '바지', '목도리', '드레스', '니트', '정장', '장갑', '가죽', '타올', '빅토리아시크릿', '엘르']
gift_dict["bath"] = ['록시땅', '샴푸', '립밤', '바디워시', '배쓰', '캐스키드슨', '바디', '바디샵', '바디로션', '더바디샵', '러쉬', '로션', '치약', '비누', '샤워젤', '이솝', '버츠비', '컨디셔너', '비욘드', '히말라야', '케라시스', '트리트먼트', '입욕제', '바디세트', '시어버터', '린스', '아베다', '사봉', '캐드키드슨', '바디케어', '핑크솔트', '칫솔', '바디미스트', '배쓰밤', '바디스크럽', '요아럽', 'aesop', '바디판타지', '워시', '헤어', '바디크림', '리엔', '쿤달', '바디오일', '루치펠로', '샤워볼', '바디밤', '바디클렌저', '바디용품', '샤워', '클렌징폼', '해피바스', '섹스밤', '올가', '인덜전트']
gift_dict["nutri"] = ['콜라겐', '락토핏', '비타민', '멀티', '생유산균']
gift_dict["leisure"] = ['골프공', '골프', '아이언', '골프채', '볼빅', '골프백', '캐디백', '캠핑', '애벌레', '캘러웨이', '미즈노', '아이언세트', '테일러메이드', '드라이버', '볼마커', '타이틀리스트']
gift_dict["baby"] = ['신생아', '아기', '베베', '딸랑이', '출산선물세트', '여아', '유아','아동',  '임산부', '아기옷',  '출산선물', '장난감', '배냇저고리', '손싸개', '턱받이', '베이비', '아가방', '딸랑이세트', '속싸개', '어린이', '출산', '남아', '바디수트', '바디슈트', '오가닉맘', '젖병', '키즈', '이유식', '모빌', '치발기', '발싸개', '의자', '아기띠', '배넷저고리', '뽀로로', '맘스네이처', '상하복', '유모차', '밤부베베', '한스펌킨', '아토팜', '타요', '주방놀이', '압소바', '라마즈', '에뜨와', '밍크뮤']
gift_dict["luxury"] = ['샤넬', '구찌', '몽블랑', '루이비통', '불가리', '버버리', '페라가모', '에르메스']
gift_dict["electric"] = ['키보드', '인덕션', '마우스', '무선', '블루투스', '마사지기', '기펠', 'pro', '트리오', '닌텐도', 'led', '삼성', '청소기', '바이마르', '갤럭시', '프라엘', '스위치', '로지텍', '커피머신', '드롱기']
gift_dict["coupon"] = ['기프티콘', '선물하기', '쿠폰']
gift_dict["furnishing"] = ['이케아', '전등', '인테리어', '한샘', '무드등', '테이블', '모이몰른', '우산', '모던하우스', '이불', '베개', '미니워머', '필로우']
# gift_dict["etc"] = ['쇼핑백', '미니', '스페셜', '홀더', '듀오', '모자이크', '위드오가닉', '화이트', '클래식', '고급', '블랙', '닥스', '여성', '핑크', '로즈', '유기농', '유리', '컬렉션', '오가닉', '홈세트', '디즈니', '골드', '카카오프렌즈', '크리스마스', '에디션', '한정판', '명절선물', '크리스탈', '올인원', '볼펜', '인형', '카카오', '일본', '빈티지', '다이어리', '케이스', '라이언', '어피치', '파머스', '레드', '울트라', '러블리', '오리지널', '스페셜세트', '그린', '단호', '한국', '라미', '레트로', '물걸레', '벨벳']

# 18





all_list = []
thanks_list = []
normal_list = []
for i in list(gift_dict.values()):
    for j in i:
        normal_list.append(j)
        
for i in list(thanks_dict.values()):
    for j in i:
        thanks_list.append(j)
        
all_list = thanks_list+normal_list
len(all_list)

578

In [24]:
# 딕셔너리 안에 포함되지 않은 단어들 

not_list = []
for i in cv.get_feature_names():
    if i not in all_list:
        not_list.append(i)
print(len(not_list))
print(list(not_list))

585
['00', '000', '000원', '00원', '0g', '0ml', '10', '100', '100ml', '100호', '10p', '10개', '10호', '11', '11호', '12', '12m', '12개', '12개입', '12종', '12호', '13', '13호', '14', '14p', '14호', '15', '15호', '16', '16호', '18', '19', '1개', '1세트', '1인', '1호', '20', '200', '200g', '200ml', '2021', '21', '21호', '22', '22호', '23', '24', '25', '250ml', '27호', '28', '2p', '2개', '2개세트', '2병', '2세트', '2인', '2인세트', '2인조', '2종', '2종세트', '2호', '30', '30ml', '30포', '36', '37', '3p', '3개', '3세트', '3종', '3종세트', '3호', '40', '4p', '4개', '4인', '4종', '4종세트', '4호', '50', '500', '500ml', '50ml', '57호', '5kg', '5ml', '5p', '5개', '5인', '5종', '5종세트', '5호', '60', '6p', '6개', '6종', '6종세트', '6호', '70', '75', '75ml', '7개', '7종', '7호', '80', '80사이즈', '8k호', '8개', '8종', '8호', '90', '9개', '9호', 'al', 'am', 'an', 'ap', 'ar', 'at', 'a호', 'bo', 'ch', 'cm', 'co', 'di', 'ea', 'ed', 'el', 'en', 'er', 'es', 'e호', 'f호', 'g×', 'g호', 'ic', 'il', 'in', 'it', 'kg', 'la', 'le', 'lo', 'ma', 'ml', 'mm', 'nh', 'n호', 'ol', 'on', 'or', 'pc', '

## 딕셔너리 단어별 매물 수 

In [25]:
thanks_one_line = "|".join(thanks_list)

one_line ="|".join(all_list)
one_line

'스팸|리챔|스팸선물세트|로스팜|스팸세트|스팸8호|햄|스팸6호|스팸3호|스팸1호|그릭슈바인|런천미트|수제햄|의성마늘|스팸복합1호|스팸선물셋트|동원|동원참치|참치|동원선물세트|사조|참치선물세트|동원참치선물세트|참치캔|행호|핸드크림|핸드|핸드워시|손소독제|더프트앤도프트|랩신|레저렉션|견과류|견과|멸치|광천김|멸치선물세트|카놀라유|참기름|식용유|해바라기씨유|트러플|들기름|올리브유|해바라기유|고급유|포도씨유|올리브오일|기름|요리유|오일세트|정관장|홍삼|인삼|홍삼정|6년근|다보록|한삼인|활기력|한뿌리|홍삼스틱|홍삼원|홍삼선물세트|청정원|cj|해표|cj제일제당|목우촌|백설|스팸복합|현명한선택|lg|튜나리챔|복합|cj선물세트|안심특선|특별한선택|롯데|특별한|제일제당|오뚜기|농협|혼합|현명한|사조해표|스팸고급유|청정원선물세트|복합1호|해표선물세트|씨제이|풍성한|육포|한우|사과|표고버섯|과일|버섯|곶감|배|한과|초콜릿|쿠키|과자|올리고당|천일염|벌꿀|미엘드|간장|식초|lg생활건강|아모레퍼시픽|애경|생활건강|엘지생활건강|아모레|엘지|생활의|마스크|섬유유연제|세제|스타벅스|오설록|차|커피|티세트|커피잔세트|이디야|카누|티백|드립백|tea|쌍계명차|에스프레소|할리스|맥심|담터|투썸|녹차|티박스|네스프레소|아메리카노|핸드드립|twg|허브티|꽃차|타바론|홍차|차세트|더치커피|비니스트|홈카페|투썸플레이스|와인|신앙촌|양키캔들|향초|향수|캔들|디퓨저|조말론|모링가|캔들워머|퍼퓸|코코도르|필로우미스트|방향제|베르사체|디퓨져|딥디크|아로마|워머세트|양키|바디필로우|더블유드레스룸|우드윅|라벤더|딥티크|랑방|보티브|자쉐이드|자캔들|화이트머스크|화장품|자음생|설화수|미스트|크림|키엘|튼살크림|프레쉬|에센스|헤라|스킨|세럼|토너|립스틱|에스티로더|스킨케어|비오템|쿠션|스틸라|브러쉬|탬버린즈|입생로랑|달바|암웨이|수분크림|오휘|디올|앰플|꼬달리|fresh|바비브라운|아이크림|논픽션|자음2종|클렌저|ahc|화장품세트|미샤|자음|마스크팩|클라란스|랑콤|버베나|코리아나|틴트|아이오페|스크럽|립밤세트|베네피

In [26]:
# 9월
septem = df[df.upload_ym == '2021-09'].reset_index(drop=False)
print("전체 매물 수:", septem.title.count())
print("-"*40)
print("명절 선물 수 :", septem[septem.title.str.contains(thanks_one_line)].title.count())
print("명절 선물 비율 :", 
      round(septem[septem.title.str.contains(thanks_one_line)].title.count() / septem.title.count() * 100, 2), "%")
print("-"*40)
print("명절+노말 리스트에 속하지 않는 매물 수 :", septem[~septem.title.str.contains(one_line)].title.count())
print("제외 매물 비율 :", round(septem[~septem.title.str.contains(one_line)].title.count() / septem.title.count() * 100, 2), "%")


전체 매물 수: 20970
----------------------------------------
명절 선물 수 : 9113
명절 선물 비율 : 43.46 %
----------------------------------------
명절+노말 리스트에 속하지 않는 매물 수 : 3628
제외 매물 비율 : 17.3 %


In [27]:
# 6월

june = df[df.upload_ym == '2021-06'].reset_index(drop=False)
# 9월
print("전체 매물 수:", june.title.count())
print("-"*40)
print("명절 선물 수 :", june[june.title.str.contains(thanks_one_line)].title.count())
print("명절 선물 비율 :", 
      round(june[june.title.str.contains(thanks_one_line)].title.count() / june.title.count() * 100, 2), "%")
print("-"*40)
print("명절+노말 리스트에 속하지 않는 매물 수 :", june[~june.title.str.contains(one_line)].title.count())
print("제외 매물 비율 :", round(june[~june.title.str.contains(one_line)].title.count() / june.title.count() * 100, 2), "%")


전체 매물 수: 3945
----------------------------------------
명절 선물 수 : 632
명절 선물 비율 : 16.02 %
----------------------------------------
명절+노말 리스트에 속하지 않는 매물 수 : 751
제외 매물 비율 : 19.04 %


# BPE 및 딕셔너리 적용 토큰화
## 토큰화 결과 비교
- 토큰화 진행시 상품 분류에 적합하지 않는 키워드도 같이 추출되므로 이를 제외하고 딕셔너리 안에 있는 단어만 추출하는 함수를 따로 적용

In [82]:
# 딕셔너리 활용 
def tokenizer_in_dict(text):
    enc = bpe.encode(text)
    for token in enc.tokens:
        token = token.replace('</w>', '')
        if token in all_list:
#             if len(token) > 1:
            yield token

    
# 제품명은 걸러내는 함수 
def tokenizer_notin_dict(text):
    enc = bpe.encode(text)
    for token in enc.tokens:
        token = token.replace('</w>', '')
        if token not in all_list:
            if len(token) > 1:
                yield token
                
def compare_bpe(num):
    text = df.title[num]
    print('article title:' , text)
    print('-'*60)
    print('bpe tokenizer:', list(tokenizer(text)))
    print('bpe+dict tokenizer:', list(tokenizer_in_dict(text)))

In [84]:
compare_bpe(1849)   # 필요한 키워드만 추출됨

article title: 양키캔들 보티브 향초 3종 투명 캔들 홀더 선물세트
------------------------------------------------------------
bpe tokenizer: ['양키캔들', '보티브', '향초', '3종', '투명', '캔들', '홀더', '선물세트']
bpe+dict tokenizer: ['양키캔들', '보티브', '향초', '캔들']


In [34]:
compare_bpe(62345) 

article title: 데톨 세정비누 세트(새제품)
------------------------------------------------------------
bpe tokenizer: ['세정', '비누', '세트', '새제품']
bpe+dict tokenizer: ['비누']


In [32]:
compare_bpe(63923)

article title: (새상품) 오설록 오땡큐 티박스 선물세트
------------------------------------------------------------
bpe tokenizer: ['새상품', '오설록', '땡큐', '티박스', '선물세트']
bpe+dict tokenizer: ['오설록', '티박스']


## 빈도수 상위 단어 재추출 (명절 빛 비명절 시즌 비교)

In [38]:
cv4 = CountVectorizer(max_features=700, tokenizer=tokenizer_in_dict)
dtm4 = cv4.fit_transform(df.title)

wc4 = pd.DataFrame({'단어': cv4.get_feature_names(), '빈도':dtm4.sum(axis=0).flat})
wc4.sort_values('빈도', ascending=False).head(30)

,단어,빈도
287,스팸,3965
549,핸드크림,2525
122,록시땅,2199
339,양키캔들,2115
87,동원,1594
6,cj,1396
430,청정원,1369
252,샴푸,1245
170,바디,1141
420,차,1090


In [39]:
wc4.to_excel('wc_578.xlsx')

### 딕셔너리안에 있는 단어로만 - 9월
- 스팸, 참치, 핸드크림, 록시땅 등이 눈에 띈다

In [35]:
cv2 = CountVectorizer(max_features=700, tokenizer=tokenizer_in_dict)
dtm2 = cv2.fit_transform(septem.title)

wc2 = pd.DataFrame({'단어': cv2.get_feature_names(), '빈도':dtm2.sum(axis=0).flat})
wc2.sort_values('빈도', ascending=False).head(30)

,단어,빈도
285,스팸,1552
6,cj,700
87,동원,642
428,청정원,570
547,핸드크림,519
121,록시땅,408
197,배,340
562,홍삼,332
421,참치,319
89,동원참치,299


### 딕셔너리안에 있는 단어로만 - 6월 (비명절 시즌)

In [36]:
june = df[df.upload_ym == '2021-06'].reset_index(drop=False)
cv4 = CountVectorizer(max_features=700, tokenizer=tokenizer_in_dict)
dtm4 = cv4.fit_transform(june.title)

wc4 = pd.DataFrame({'단어': cv4.get_feature_names(), '빈도':dtm4.sum(axis=0).flat})
wc4.sort_values('빈도', ascending=False).head(30)

,단어,빈도
521,핸드크림,133
113,록시땅,131
317,양키캔들,118
160,바디,67
239,샴푸,65
395,차,59
445,크림,59
272,스팸,56
266,스타벅스,56
431,컵,55


### 딕셔너리에 포함되지 않은 단어로만
- 새상품, 미개봉, 세제품, 미사용 이라는 단어가 눈에 띈다

In [37]:
cv3 = CountVectorizer(max_features=700, tokenizer=tokenizer_notin_dict)
dtm3 = cv3.fit_transform(df.title)

wc3 = pd.DataFrame({'단어': cv3.get_feature_names(), '빈도':dtm3.sum(axis=0).flat})
wc3.sort_values('빈도', ascending=False).head(20)

,단어,빈도
408,선물세트,24645
426,세트,19765
398,새상품,14478
399,새제품,6207
341,미개봉,5182
649,팝니다,3792
404,선물,3129
646,판매합니다,2696
429,셋트,1798
648,팔아요,1534


# 토큰화 진행

In [25]:
df['token'] = df.title.apply(lambda x : sorted(list(set(list(tokenizer_in_dict(x))))))

In [26]:
df.shape

(84102, 16)

In [27]:
df_token = df[df.token.apply(len) > 0].reset_index(drop=True)

In [28]:
df_token.shape

(67421, 16)

In [29]:
df_token['token_sum'] = df_token.token.apply(lambda x : " ".join([i for i in x]))

# 카테고리 컬럼

In [30]:
gift_cate_list = list(gift_dict.keys())
thanks_cate_list = list(thanks_dict.keys())

def category_sort(x):
    category_list = []
    for token in x:
        for name in thanks_cate_list:
            if token in thanks_dict[name]:
                category_list.append(name)
        for name in gift_cate_list:
            if token in gift_dict[name]:
                category_list.append(name)
    return sorted(list(set(category_list)))



In [31]:
df_token['cate_temp'] = df_token['token'].apply(category_sort)

In [32]:
def category_final(x):
    if len(x) == 1:
        result = x[0]
    elif len(x) > 1:
        if 'hand' in x:
            result = 'hand'
        elif 'etc' in x:
            x.remove('etc')
            result = x[0]
        elif 'tools' in x:
            result = 'tools'
        elif ('oil' in x)\
            or ('spam' in x)\
            or ('tuna' in x):
            result = 'mix'
        elif 'man' in x:
            result = 'man'
        elif 'cosmetics' in x:
            result = 'cosmetics'
        elif 'ginseng' in x:
            result = 'ginseng'
        elif 'bath_thanks' in x:
            result = 'bath_thanks'
        elif ('coffee' in x) \
            and ('dishes' in x):
            result = 'dishes'
        elif 'kitchen' in x:
            result = 'kitchen'
        elif ('cloth' in x) \
            and ('luxury' in x):
            result = 'luxury'
        elif 'perfume' in x :
            result = 'perfume'
        else:
            result = x[0]
    return result 


In [33]:
df_token['cate_final'] = df_token.cate_temp.apply(category_final)

In [35]:
df_token[(df_token.cate_temp.apply(len) > 1)][['title', 'token', 'cate_temp',  'cate_final']].sample(20)

,title,token,cate_temp,cate_final
61179,동원 선물세트(참치8 + 리챔4),"[동원, 리챔, 참치]","[spam, tuna]",mix
34521,모로칸오일 새제품 선물세트 헤어오일 헤어에센스,"[에센스, 헤어]","[bath, cosmetics]",cosmetics
17230,록시땅 핸드크림 30ml 2개 세트,"[록시땅, 핸드크림]","[bath, hand]",hand
57681,아기 유아옷 여아원피스 90사이즈,"[아기, 여아, 원피스, 유아]","[baby, cloth]",baby
29646,한국전통 고려청자 도자기 장식용 인테리어 소품 01,"[도자기, 인테리어]","[dishes, furnishing]",dishes
35759,새상품(세탁만 함) 카터스 아기옷 12m + 모자,"[모자, 아기옷]","[baby, cloth]",baby
9851,스팸식용유올리고당 선물세트,"[스팸, 식용유, 올리고당]","[oil, spam, spicy]",mix
51582,"(미개봉새상품) 기펠 엘리시아 회전조리도구 10,000원","[기펠, 조리도구]","[electric, kitchen]",kitchen
37612,"동원 참치,햄 선물셋트","[동원, 참치, 햄]","[spam, tuna]",mix
45181,록시땅 시어버터 핸드크림 + 립밤 듀오 세트 (선물 포장),"[록시땅, 립밤, 시어버터, 핸드크림]","[bath, hand]",hand


In [208]:
df_token.groupby(['region_gu','cate_final', ]).title.count()

region_gu  cate_final 
강남구        baby           252
           bath           337
           bath_thanks     52
           cloth          236
           coffee         249
                         ... 
중랑구        perfume        134
           snacks           9
           spam           153
           spicy           11
           tuna            52
Name: title, Length: 800, dtype: int64

In [36]:
df_token[df_token.cate_final == 'fruits'][['title', 'token', 'cate_temp', 'cate_final']].sample(40)

,title,token,cate_temp,cate_final
12598,썬플러스 사과 선물세트 5kg(15과/선물포장) 추석 선물 세트 일요일까지만,[사과],[fruits],fruits
17797,사과선물세트 5kg 16과,[사과],[fruits],fruits
29187,추석선물세트 7.5kg 배 특등급,[배],[fruits],fruits
32122,🍄송화버섯🍄,[버섯],[fruits],fruits
48752,디바총\코스프레\보조배터리,[배],[fruits],fruits
61609,사과 선물세트,[사과],[fruits],fruits
57912,참송고 추재버섯 선물세트,[버섯],[fruits],fruits
32326,사과 선물세트 팝니다,[사과],[fruits],fruits
40384,배선물세트,[배],[fruits],fruits
8141,선물용과일,[과일],[fruits],fruits


In [37]:
df_token.to_excel('df_token_2.xlsx', index=False)